# GSM CLI Jupyter Integration Examples

This notebook demonstrates how to use the GSM CLI interface from within Jupyter for:
- Dynamic parameter specification retrieval
- Interactive parameter validation
- Simulation execution and result visualization
- Network communication with remote GSM servers

## Features Demonstrated

1. **Dynamic Parameter Discovery**: Retrieve parameter specifications for different GSM models
2. **Interactive Validation**: Use widgets for parameter input and real-time validation
3. **Simulation Execution**: Run simulations and visualize results
4. **Network Integration**: Connect to remote GSM servers for distributed computing
5. **Batch Processing**: Execute multiple simulations with parameter variations

## Requirements

```bash
pip install ipywidgets matplotlib plotly requests
```

In [1]:
# Import Required Libraries
import os
import sys
import subprocess
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import requests
import time
from typing import Dict, List, Any, Optional

# Widget imports for interactive elements
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, JSON
    WIDGETS_AVAILABLE = True
except ImportError:
    print("Warning: ipywidgets not available. Install with: pip install ipywidgets")
    WIDGETS_AVAILABLE = False

# Plotting imports
try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    print("Warning: plotly not available. Install with: pip install plotly")
    PLOTLY_AVAILABLE = False

# Set up paths
CURRENT_DIR = Path.cwd()
CLI_PATH = CURRENT_DIR.parent / "cli_gsm.py"
EXAMPLES_DIR = CURRENT_DIR

print(f"Current directory: {CURRENT_DIR}")
print(f"CLI path: {CLI_PATH}")
print(f"Examples directory: {EXAMPLES_DIR}")
print(f"Widgets available: {WIDGETS_AVAILABLE}")
print(f"Plotly available: {PLOTLY_AVAILABLE}")

Current directory: /home/rch/Coding/bmcs_matmod/bmcs_matmod/gsm_lagrange/examples
CLI path: /home/rch/Coding/bmcs_matmod/bmcs_matmod/gsm_lagrange/cli_gsm.py
Examples directory: /home/rch/Coding/bmcs_matmod/bmcs_matmod/gsm_lagrange/examples
Widgets available: True
Plotly available: True


In [2]:
# GSM CLI Helper Class for Jupyter Integration

class GSMJupyterInterface:
    """Helper class for GSM CLI integration in Jupyter notebooks"""
    
    def __init__(self, cli_path: str = "../cli_gsm.py"):
        self.cli_path = cli_path
        self.server_url = "http://localhost:8888"
        
    def run_cli_command(self, args: List[str], timeout: int = 30) -> Dict[str, Any]:
        """Run a CLI command and return the result"""
        try:
            cmd = ["python", self.cli_path] + args
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
            
            if result.returncode == 0:
                try:
                    # Try to parse JSON output
                    return json.loads(result.stdout)
                except json.JSONDecodeError:
                    # Return raw text if not JSON
                    return {"status": "success", "output": result.stdout}
            else:
                return {"status": "error", "error": result.stderr}
                
        except subprocess.TimeoutExpired:
            return {"status": "error", "error": "Command timeout"}
        except Exception as e:
            return {"status": "error", "error": str(e)}
    
    def list_models(self) -> Dict[str, Any]:
        """Get list of available models"""
        return self.run_cli_command(["--list-models", "--json-output"])
    
    def get_parameter_spec(self, model_name: str) -> Dict[str, Any]:
        """Get parameter specification for a model"""
        return self.run_cli_command(["--get-param-spec", model_name, "--json-output"])
    
    def validate_parameters(self, model: str, formulation: str, 
                          parameters: Dict, loading: Dict) -> Dict[str, Any]:
        """Validate parameters"""
        params_json = json.dumps(parameters)
        loading_json = json.dumps(loading)
        
        return self.run_cli_command([
            "--model", model,
            "--formulation", formulation,
            "--params-inline", params_json,
            "--loading-inline", loading_json,
            "--validate-only",
            "--json-output"
        ])
    
    def execute_simulation(self, model: str, formulation: str,
                          parameters: Dict, loading: Dict, 
                          config: Dict = None) -> Dict[str, Any]:
        """Execute a simulation"""
        params_json = json.dumps(parameters)
        loading_json = json.dumps(loading)
        
        args = [
            "--model", model,
            "--formulation", formulation,
            "--params-inline", params_json,
            "--loading-inline", loading_json,
            "--json-output"
        ]
        
        if config:
            config_json = json.dumps(config)
            args.extend(["--config-inline", config_json])
        
        return self.run_cli_command(args)

# Initialize the interface
gsm = GSMJupyterInterface()
print("GSM Jupyter Interface initialized")

GSM Jupyter Interface initialized


## 1. Dynamic Model Discovery

Let's start by discovering what GSM models are available and examining their parameter specifications.

In [3]:
# Discover available GSM models
models_result = gsm.list_models()

if models_result["status"] == "success":
    print("Available GSM Models:")
    print("=" * 50)
    
    # Create a DataFrame for better display
    model_data = []
    for model in models_result.get("models", []):
        if isinstance(model, dict):
            model_data.append({
                "Model": model.get("name", "Unknown"),
                "Mechanism": model.get("mechanism", "Unknown"),
                "Description": model.get("description", "No description")
            })
        else:
            model_data.append({
                "Model": model,
                "Mechanism": model.replace("GSM1D_", "") if model.startswith("GSM1D_") else "Unknown",
                "Description": "Available model"
            })
    
    if model_data:
        df_models = pd.DataFrame(model_data)
        display(df_models)
        
        # Store model names for later use
        available_models = [row["Model"] for row in model_data]
        print(f"\nFound {len(available_models)} models: {', '.join(available_models)}")
    else:
        print("No models found in the response")
        available_models = []
        
else:
    print(f"Error retrieving models: {models_result.get('error', 'Unknown error')}")
    available_models = []

Available GSM Models:


,Model,Mechanism,Description
0,GSM1D_ED,ED,Elasto-Damage Model
1,GSM1D_EP,EP,Elasto-Plastic Model
2,GSM1D_EPD,EPD,Elasto-Plastic-Damage Model
3,GSM1D_EVP,EVP,Elasto-Visco-Plastic Model
4,GSM1D_EVPD,EVPD,Elasto-Visco-Plastic-Damage Model
5,GSM1D_VE,VE,Visco-Elastic Model
6,GSM1D_VED,VED,Visco-Elasto-Damage Model
7,GSM1D_VEVP,VEVP,Visco-Elasto-Visco-Plastic Model
8,GSM1D_VEVPD,VEVPD,Visco-Elasto-Visco-Plastic-Damage Model



Found 9 models: GSM1D_ED, GSM1D_EP, GSM1D_EPD, GSM1D_EVP, GSM1D_EVPD, GSM1D_VE, GSM1D_VED, GSM1D_VEVP, GSM1D_VEVPD


## 2. Parameter Specification Exploration

Now let's examine the parameter specifications for different models. This is crucial for understanding what parameters each model requires and their valid ranges.

In [4]:
# Explore parameter specifications for key models
key_models = ["GSM1D_ED", "GSM1D_EP", "GSM1D_VE"] if available_models else []

parameter_specs = {}

for model in key_models:
    if model in available_models:
        print(f"\n{'='*60}")
        print(f"Parameter Specification for {model}")
        print(f"{'='*60}")
        
        spec_result = gsm.get_parameter_spec(model)
        
        if spec_result["status"] == "success":
            spec = spec_result["specification"]
            parameter_specs[model] = spec
            
            print(f"Model: {spec['model_name']}")
            print(f"Parameters ({len(spec['parameters'])}):")
            
            # Create parameter table
            param_data = []
            for param_name, param_info in spec["parameters"].items():
                desc = spec["parameter_descriptions"].get(param_name, "No description")
                bounds = spec["parameter_bounds"].get(param_name, "No bounds")
                units = spec["parameter_units"].get(param_name, "-")
                
                param_data.append({
                    "Parameter": param_name,
                    "Description": desc,
                    "Type": param_info["type"],
                    "Required": param_info["required"],
                    "Units": units,
                    "Bounds": str(bounds)
                })
            
            df_params = pd.DataFrame(param_data)
            display(df_params)
            
        else:
            print(f"Error getting specification: {spec_result.get('error', 'Unknown error')}")

print(f"\nParameter specifications retrieved for {len(parameter_specs)} models")


Parameter Specification for GSM1D_ED
Model: GSM1D_ED
Parameters (0):
Model: GSM1D_ED
Parameters (0):


""



Parameter Specification for GSM1D_EP
Model: GSM1D_EP
Parameters (0):
Model: GSM1D_EP
Parameters (0):


""



Parameter Specification for GSM1D_VE
Model: GSM1D_VE
Parameters (0):
Model: GSM1D_VE
Parameters (0):


""



Parameter specifications retrieved for 3 models


## 3. Interactive Parameter Validation

Let's create an interactive interface for parameter validation using widgets (if available). This allows real-time validation as parameters are adjusted.

In [5]:
# Interactive Parameter Validation Widget

if WIDGETS_AVAILABLE and parameter_specs:
    
    class ParameterValidator:
        def __init__(self, gsm_interface, model_specs):
            self.gsm = gsm_interface
            self.specs = model_specs
            self.widgets = {}
            self.output = widgets.Output()
            
        def create_parameter_widgets(self, model_name):
            """Create widgets for model parameters"""
            if model_name not in self.specs:
                return []
            
            spec = self.specs[model_name]
            widget_list = []
            
            for param_name, param_info in spec["parameters"].items():
                bounds = spec["parameter_bounds"].get(param_name, (0, 100))
                default_val = bounds[0] + (bounds[1] - bounds[0]) * 0.5 if isinstance(bounds, tuple) else 1.0
                
                widget = widgets.FloatSlider(
                    value=default_val,
                    min=bounds[0] if isinstance(bounds, tuple) else 0,
                    max=bounds[1] if isinstance(bounds, tuple) else 100,
                    step=0.1,
                    description=param_name,
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='400px')
                )
                
                widget_list.append(widget)
                self.widgets[param_name] = widget
                
            return widget_list
        
        def validate_current_parameters(self, model_name, formulation="F"):
            """Validate current parameter values"""
            if model_name not in self.specs:
                return
                
            parameters = {name: widget.value for name, widget in self.widgets.items()}
            
            # Simple loading for validation
            loading = {
                "time_array": [0.0, 0.5, 1.0],
                "strain_history": [0.0, 0.005, 0.01],
                "loading_type": "strain_controlled"
            }
            
            result = self.gsm.validate_parameters(model_name, formulation, parameters, loading)
            
            with self.output:
                self.output.clear_output()
                print(f"Validation for {model_name}:")
                print("-" * 40)
                
                if result["status"] == "validation_complete":
                    if result["valid"]:
                        print("✅ Parameters are VALID")
                        print(f"Parameters: {parameters}")
                    else:
                        print("❌ Parameters are INVALID")
                        if "errors" in result:
                            print("Errors:")
                            for error in result["errors"]:
                                print(f"  - {error}")
                else:
                    print(f"Validation error: {result.get('error', 'Unknown error')}")
    
    # Create validator instance
    validator = ParameterValidator(gsm, parameter_specs)
    
    # Model selection widget
    model_dropdown = widgets.Dropdown(
        options=list(parameter_specs.keys()),
        value=list(parameter_specs.keys())[0] if parameter_specs else None,
        description='Model:',
        style={'description_width': 'initial'}
    )
    
    # Formulation selection
    formulation_dropdown = widgets.Dropdown(
        options=['F', 'G', 'Helmholtz', 'Gibbs'],
        value='F',
        description='Formulation:',
        style={'description_width': 'initial'}
    )
    
    # Create parameter widgets for initial model
    if parameter_specs:
        initial_model = list(parameter_specs.keys())[0]
        param_widgets = validator.create_parameter_widgets(initial_model)
        
        # Validate button
        validate_button = widgets.Button(
            description='Validate Parameters',
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
        
        def on_validate_click(b):
            validator.validate_current_parameters(
                model_dropdown.value, 
                formulation_dropdown.value
            )
        
        def on_model_change(change):
            # Recreate parameter widgets for new model
            new_widgets = validator.create_parameter_widgets(change['new'])
            param_container.children = new_widgets
        
        validate_button.on_click(on_validate_click)
        model_dropdown.observe(on_model_change, names='value')
        
        # Layout
        param_container = widgets.VBox(param_widgets)
        controls = widgets.HBox([model_dropdown, formulation_dropdown, validate_button])
        
        ui = widgets.VBox([
            widgets.HTML("<h3>Interactive Parameter Validation</h3>"),
            controls,
            param_container,
            validator.output
        ])
        
        display(ui)
        
        # Initial validation
        validator.validate_current_parameters(initial_model)
        
else:
    print("Interactive widgets not available or no parameter specifications loaded.")
    print("You can still validate parameters manually:")
    
    # Manual validation example
    if available_models:
        example_model = available_models[0]
        example_params = {
            "E": 30000.0,
            "S": 1.0,
            "c": 2.0,
            "r": 0.5,
            "eps_0": 0.001
        }
        
        example_loading = {
            "time_array": [0.0, 0.5, 1.0],
            "strain_history": [0.0, 0.005, 0.01],
            "loading_type": "strain_controlled"
        }
        
        validation_result = gsm.validate_parameters(example_model, "F", example_params, example_loading)
        print(f"\nExample validation for {example_model}:")
        print(json.dumps(validation_result, indent=2))

## 4. Simulation Execution and Visualization

Now let's execute some simulations and visualize the results. We'll compare different material models and loading scenarios.

In [6]:
# Execute simulations with different models and parameters

def create_loading_scenario(scenario_type="monotonic", max_strain=0.01, n_steps=11):
    """Create different loading scenarios"""
    time_array = np.linspace(0, 1.0, n_steps)
    
    if scenario_type == "monotonic":
        strain_history = np.linspace(0, max_strain, n_steps)
    elif scenario_type == "cyclic":
        # Simple tension-compression cycle
        strain_history = max_strain * np.sin(2 * np.pi * time_array)
    elif scenario_type == "loading_unloading":
        # Load to max, then unload
        mid_point = n_steps // 2
        strain_history = np.concatenate([
            np.linspace(0, max_strain, mid_point),
            np.linspace(max_strain, 0, n_steps - mid_point)
        ])
    else:
        strain_history = np.linspace(0, max_strain, n_steps)
    
    return {
        "time_array": time_array.tolist(),
        "strain_history": strain_history.tolist(),
        "loading_type": "strain_controlled"
    }

# Define simulation scenarios
simulation_scenarios = [
    {
        "name": "GSM1D_ED_Monotonic",
        "model": "GSM1D_ED",
        "parameters": {"E": 30000.0, "S": 1.0, "c": 2.0, "r": 0.5, "eps_0": 0.001},
        "loading": create_loading_scenario("monotonic", 0.01, 11),
        "formulation": "F"
    },
    {
        "name": "GSM1D_ED_LoadUnload",
        "model": "GSM1D_ED",
        "parameters": {"E": 30000.0, "S": 1.0, "c": 2.0, "r": 0.5, "eps_0": 0.001},
        "loading": create_loading_scenario("loading_unloading", 0.01, 21),
        "formulation": "F"
    }
]

# Add more scenarios if other models are available
if "GSM1D_EP" in available_models:
    simulation_scenarios.append({
        "name": "GSM1D_EP_Monotonic",
        "model": "GSM1D_EP",
        "parameters": {"E": 35000.0, "S": 1.5, "K": 25000.0, "G": 14000.0, "eta_vp": 100.0},
        "loading": create_loading_scenario("monotonic", 0.008, 11),
        "formulation": "F"
    })

# Execute simulations
simulation_results = []

print("Executing simulations...")
print("=" * 50)

for i, scenario in enumerate(simulation_scenarios):
    print(f"\n{i+1}. Running {scenario['name']}...")
    
    start_time = time.time()
    result = gsm.execute_simulation(
        model=scenario["model"],
        formulation=scenario["formulation"],
        parameters=scenario["parameters"],
        loading=scenario["loading"]
    )
    execution_time = time.time() - start_time
    
    if result["status"] == "success":
        print(f"   ✅ Completed in {execution_time:.3f}s")
        result["scenario_name"] = scenario["name"]
        result["execution_time_jupyter"] = execution_time
        simulation_results.append(result)
        
        # Quick result summary
        if "response" in result:
            response = result["response"]
            final_strain = response["eps_t"][-1] if "eps_t" in response else "N/A"
            final_stress = response["sig_t"][-1] if "sig_t" in response else "N/A"
            print(f"   Final strain: {final_strain:.6f}, Final stress: {final_stress:.2f} MPa")
        
    else:
        print(f"   ❌ Failed: {result.get('error', 'Unknown error')}")

print(f"\nCompleted {len(simulation_results)} successful simulations out of {len(simulation_scenarios)} total")

Executing simulations...

1. Running GSM1D_ED_Monotonic...
   ✅ Completed in 9.478s

2. Running GSM1D_ED_LoadUnload...
   ✅ Completed in 9.478s

2. Running GSM1D_ED_LoadUnload...
   ✅ Completed in 9.405s

3. Running GSM1D_EP_Monotonic...
   ✅ Completed in 9.405s

3. Running GSM1D_EP_Monotonic...
   ✅ Completed in 9.605s

Completed 3 successful simulations out of 3 total
   ✅ Completed in 9.605s

Completed 3 successful simulations out of 3 total


In [7]:
# Visualize simulation results

if simulation_results:
    
    # Create stress-strain plots
    if PLOTLY_AVAILABLE:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Stress vs Time', 'Strain vs Time', 'Stress-Strain', 'Internal Variables'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        colors = px.colors.qualitative.Plotly
        
        for i, result in enumerate(simulation_results):
            if "response" in result:
                response = result["response"]
                name = result["scenario_name"]
                color = colors[i % len(colors)]
                
                # Extract data
                time = np.array(response.get("t_t", []))
                strain = np.array(response.get("eps_t", []))
                stress = np.array(response.get("sig_t", []))
                
                # Stress vs Time
                fig.add_trace(
                    go.Scatter(x=time, y=stress, name=f"{name} - Stress", 
                              line=dict(color=color), showlegend=True),
                    row=1, col=1
                )
                
                # Strain vs Time
                fig.add_trace(
                    go.Scatter(x=time, y=strain, name=f"{name} - Strain", 
                              line=dict(color=color, dash='dash'), showlegend=True),
                    row=1, col=2
                )
                
                # Stress-Strain
                fig.add_trace(
                    go.Scatter(x=strain, y=stress, name=f"{name} - σ-ε", 
                              line=dict(color=color), showlegend=True),
                    row=2, col=1
                )
                
                # Internal variables (if available)
                if "Eps_t_flat" in response:
                    internal_vars = np.array(response["Eps_t_flat"])
                    if internal_vars.size > 0 and len(internal_vars.shape) > 1:
                        fig.add_trace(
                            go.Scatter(x=time, y=internal_vars[:, 0], name=f"{name} - Internal", 
                                      line=dict(color=color, dash='dot'), showlegend=True),
                            row=2, col=2
                        )
        
        # Update layout
        fig.update_xaxes(title_text="Time [s]", row=1, col=1)
        fig.update_xaxes(title_text="Time [s]", row=1, col=2)
        fig.update_xaxes(title_text="Strain [-]", row=2, col=1)
        fig.update_xaxes(title_text="Time [s]", row=2, col=2)
        
        fig.update_yaxes(title_text="Stress [MPa]", row=1, col=1)
        fig.update_yaxes(title_text="Strain [-]", row=1, col=2)
        fig.update_yaxes(title_text="Stress [MPa]", row=2, col=1)
        fig.update_yaxes(title_text="Internal Variables", row=2, col=2)
        
        fig.update_layout(
            title="GSM Simulation Results Comparison",
            height=800,
            showlegend=True
        )
        
        fig.show()
        
    else:
        # Fallback to matplotlib
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        for i, result in enumerate(simulation_results):
            if "response" in result:
                response = result["response"]
                name = result["scenario_name"]
                
                time = np.array(response.get("t_t", []))
                strain = np.array(response.get("eps_t", []))
                stress = np.array(response.get("sig_t", []))
                
                # Stress vs Time
                ax1.plot(time, stress, label=f"{name}", linewidth=2)
                ax1.set_xlabel("Time [s]")
                ax1.set_ylabel("Stress [MPa]")
                ax1.set_title("Stress vs Time")
                ax1.grid(True)
                ax1.legend()
                
                # Strain vs Time
                ax2.plot(time, strain, label=f"{name}", linewidth=2, linestyle='--')
                ax2.set_xlabel("Time [s]")
                ax2.set_ylabel("Strain [-]")
                ax2.set_title("Strain vs Time")
                ax2.grid(True)
                ax2.legend()
                
                # Stress-Strain
                ax3.plot(strain, stress, label=f"{name}", linewidth=2)
                ax3.set_xlabel("Strain [-]")
                ax3.set_ylabel("Stress [MPa]")
                ax3.set_title("Stress-Strain Relationship")
                ax3.grid(True)
                ax3.legend()
                
                # Internal variables
                if "Eps_t_flat" in response:
                    internal_vars = np.array(response["Eps_t_flat"])
                    if internal_vars.size > 0 and len(internal_vars.shape) > 1:
                        ax4.plot(time, internal_vars[:, 0], label=f"{name}", linewidth=2, linestyle=':')
                
        ax4.set_xlabel("Time [s]")
        ax4.set_ylabel("Internal Variables")
        ax4.set_title("Internal Variables Evolution")
        ax4.grid(True)
        ax4.legend()
        
        plt.tight_layout()
        plt.show()
        
    # Summary table
    print("\\nSimulation Summary:")
    print("=" * 80)
    
    summary_data = []
    for result in simulation_results:
        if "response" in result:
            response = result["response"]
            final_strain = response["eps_t"][-1] if "eps_t" in response else 0
            final_stress = response["sig_t"][-1] if "sig_t" in response else 0
            exec_time = result.get("execution_time_jupyter", 0)
            
            summary_data.append({
                "Scenario": result["scenario_name"],
                "Model": result.get("model_name", "Unknown"),
                "Final Strain": f"{final_strain:.6f}",
                "Final Stress [MPa]": f"{final_stress:.2f}",
                "Exec Time [s]": f"{exec_time:.3f}"
            })
    
    if summary_data:
        df_summary = pd.DataFrame(summary_data)
        display(df_summary)
        
else:
    print("No simulation results to visualize.")

\nSimulation Summary:


## 5. Network Client Integration

The GSM CLI can also run as a network server for distributed computing. Let's demonstrate how to interact with a remote GSM server.

In [12]:
# Network Client for Remote GSM Server

class GSMNetworkClient:
    """Client for interacting with remote GSM CLI server"""
    
    def __init__(self, base_url: str = "http://localhost:8889"):
        self.base_url = base_url.rstrip('/')
        self.session = requests.Session()
        self.session.headers.update({'Content-Type': 'application/json'})
    
    def check_server_health(self) -> bool:
        """Check if the server is running"""
        try:
            response = self.session.get(f"{self.base_url}/health", timeout=5)
            return response.status_code == 200
        except requests.RequestException:
            return False
    
    def list_models(self) -> Dict[str, Any]:
        """Get list of available models from server"""
        try:
            response = self.session.get(f"{self.base_url}/models")
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            return {"status": "error", "error": str(e)}
    
    def get_parameter_spec(self, model_name: str) -> Dict[str, Any]:
        """Get parameter specification from server"""
        try:
            response = self.session.get(f"{self.base_url}/param-spec/{model_name}")
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            return {"status": "error", "error": str(e)}
    
    def execute_simulation(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute simulation on remote server"""
        try:
            response = self.session.post(f"{self.base_url}/simulate", json=request_data)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            return {"status": "error", "error": str(e)}

# Test network client (server must be running separately)
network_client = GSMNetworkClient()

print("Testing Network Client Connection...")
print("=" * 50)

# Check if server is available
server_available = network_client.check_server_health()
print(f"Server health check: {'✅ Available' if server_available else '❌ Not available'}")

if server_available:
    print("\\n🌐 Connected to remote GSM server!")
    
    # List models from server
    remote_models = network_client.list_models()
    if remote_models["status"] == "success":
        print(f"Remote models: {remote_models['models']}")
    
    # Get parameter spec from server
    if remote_models["status"] == "success" and remote_models["models"]:
        test_model = remote_models["models"][0]
        remote_spec = network_client.get_parameter_spec(test_model)
        if remote_spec["status"] == "success":
            print(f"\\nRemote parameter spec for {test_model}:")
            print(f"Parameters: {list(remote_spec['specification']['parameters'].keys())}")
    
    # Execute a remote simulation
    remote_request = {
        "model": "GSM1D_ED",
        "formulation": "F",
        "parameters": {
            "E": 30000.0,
            "S": 1.0,
            "c": 2.0,
            "r": 0.5,
            "eps_0": 0.001
        },
        "loading": {
            "time_array": [0.0, 0.5, 1.0],
            "strain_history": [0.0, 0.005, 0.01],
            "loading_type": "strain_controlled"
        }
    }
    
    print("\\nExecuting remote simulation...")
    start_time = time.time()
    remote_result = network_client.execute_simulation(remote_request)
    network_time = time.time() - start_time
    
    if remote_result["status"] == "success":
        print(f"✅ Remote simulation completed in {network_time:.3f}s")
        if "response" in remote_result:
            final_stress = remote_result["response"]["sig_t"][-1]
            print(f"Final stress from remote simulation: {final_stress:.2f} MPa")
    else:
        print(f"❌ Remote simulation failed: {remote_result.get('error', 'Unknown error')}")
        
else:
    print("\\n📝 Server not running. To test network functionality:")
    print("   1. Open a new terminal")
    print("   2. Navigate to the GSM CLI directory")
    print("   3. Run: python cli_gsm.py --serve --port 8888")
    print("   4. Re-run this cell")
    print("\\n💡 Network mode allows distributed computing across multiple machines")

Testing Network Client Connection...
Server health check: ❌ Not available
\n📝 Server not running. To test network functionality:
   1. Open a new terminal
   2. Navigate to the GSM CLI directory
   3. Run: python cli_gsm.py --serve --port 8888
   4. Re-run this cell
\n💡 Network mode allows distributed computing across multiple machines


## 6. Summary and Advanced Usage

This notebook has demonstrated the key features of the GSM CLI interface:

### ✅ What We've Covered

1. **Dynamic Model Discovery**: Automatically discover available GSM models and their capabilities
2. **Parameter Specification**: Retrieve detailed parameter requirements with bounds and descriptions
3. **Interactive Validation**: Real-time parameter validation with immediate feedback
4. **Simulation Execution**: Run simulations with different models and loading scenarios
5. **Result Visualization**: Create comprehensive plots and analysis of simulation results
6. **Network Integration**: Connect to remote GSM servers for distributed computing

### 🚀 Advanced Usage Patterns

The GSM CLI interface supports several advanced workflows:

#### Parameter Studies
```python
# Batch parameter variations
parameter_variations = {
    "E": [20000, 30000, 40000],
    "S": [0.5, 1.0, 1.5, 2.0]
}
# Process all combinations automatically
```

#### Workchain Integration
```python
# Integration with workflow managers like AiiDA
from aiida import load_profile
load_profile()
# Submit GSM simulations as AiiDA WorkChains
```

#### Remote Computing
```python
# Distribute simulations across multiple servers
servers = ["http://node1:8888", "http://node2:8888", "http://node3:8888"]
# Load balance simulations across computational nodes
```

### 📊 Key Benefits

- **Reproducibility**: All parameters and configurations are explicitly specified
- **Scalability**: Network mode enables distributed computing
- **Flexibility**: Support for multiple energy formulations and models
- **Integration**: Easy integration with Jupyter, AiiDA, and other tools
- **Validation**: Built-in parameter validation prevents invalid simulations

### 🔧 Next Steps

1. **Explore Different Models**: Try GSM1D_EP, GSM1D_VE, and other available models
2. **Custom Loading**: Create complex loading scenarios (cyclic, multi-stage, etc.)
3. **Parameter Studies**: Use the batch processing tools for sensitivity analysis
4. **Network Deployment**: Set up distributed computing infrastructure
5. **Workflow Integration**: Integrate with your existing computational workflows

### 📚 Documentation

- Check the `examples/` directory for more detailed examples
- Review the CLI help: `python cli_gsm.py --help`
- Examine the network server endpoints for API integration
- Look at the batch processing scripts for automation examples